In [1]:
import os, glob
import numpy as np
import random
import json
import copy
import time
import tensorflow as tf
import datetime


/home/kwierman/spack/opt/spack/linux-ubuntu16.04-x86_64/gcc-5.4.1/python-3.6.1-6qs3ck4umkbbao3lu5t3kslx3dr7knxs/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class TextDataSet(object):
    
    def __init__(self, filepath='../data/20news-18828', length=20000):
        self.basepath = filepath
        self.length=length
        self.class_map={}
        self.classes = os.listdir(filepath)
        for index, value in enumerate(self.classes):
            self.class_map[value] = index
        self.dataset = None
        
    def load(self, class_map, dataset):
        with open(class_map, 'r') as _file:
            self.class_map = copy.copy(json.load(_file))
        with open(dataset, 'r') as _file:
            self.dataset = copy.copy(json.load(_file))
        for cls in self.class_map:
            random.shuffle(self.dataset[str(self.class_map[cls])])
        
    def create_datasets(self):

        train = {}
        val = {}
        test ={}
        
        for i in self.classes:
            train[self.class_map[i]]=[]
            val[self.class_map[i]]=[]
            test[self.class_map[i]]=[]
            for filename in glob.glob(os.path.join(self.basepath, i, '*')):
                r = np.random.random_sample()
                if r > 0.95:
                    test[self.class_map[i]].append(filename)
                elif r > 0.9:
                    val[self.class_map[i]].append(filename)
                else:
                    train[self.class_map[i]].append(filename)
            random.shuffle(train[self.class_map[i]])
            random.shuffle(test[self.class_map[i]])
            random.shuffle(val[self.class_map[i]])
            
        with open('train.json', 'w') as output:
            json.dump(train, output)
        with open('test.json', 'w') as output:
            json.dump(test, output)
        with open('val.json', 'w') as output:
            json.dump(val, output)                
                
        with open('class_map.json', 'w') as output:
            json.dump(self.class_map, output)

    def get_text(self, filename):
        output= np.zeros(shape=(self.length,), dtype=np.integer)
        index = 0
        with open(filename, 'r', encoding='utf-8', errors='ignore') as input_file:
            for line in input_file.readlines():
                for char in line:
                    if index >= self.length:
                        break
                    output[index] = self.decode_character(char)
                    index += 1
        return output
            
    def decode_character(self, char):
        try:
            return ord(char)
        except UnicodeDecodeError:
            return 0
    
    def get_random_filenames(self):
        tmp = []
        for cls in  self.class_map:
            for i in range(1):
                try:
                    tmp.append( (self.dataset[str(self.class_map[cls])].pop(), self.class_map[cls]))
                except IndexError:
                    raise StopIteration
        random.shuffle(tmp)
        return [i[0] for i in tmp], [i[1] for i in tmp]
    
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next()
    
    def next(self):
        x, y = self.get_random_filenames()
        tmp_x = []
        for i in x:
            encoding = self.get_text(i)
            tmp_x.append(encoding)
        x = tmp_x    
        tmp_x = np.zeros(shape=(len(x), self.length))
        for index, arr in enumerate(x):
            tmp_x[index][:len(arr)] = arr
        # So, now we're going to have to create the Y matrix
        tmp_y = np.zeros(shape=(len(y),20))
        for index, value in enumerate(y):
                tmp_y[index, value] =1
        return tmp_x, tmp_y

In [3]:
class TextCNN(object):
    
    def __init__(self, batch_size=20):
        self.batch_size = batch_size
        self.X = tf.placeholder(tf.float32, [None, 20000], name="X")
        self.Y = tf.placeholder(tf.float32, [None, 20], name="Y")

        X = tf.reshape(self.X, shape=[self.batch_size, 20000, 1])
        self.a1 = self.create_conv_layer(X, 5, 3, 128, 1)
        width = int((20000-5)/3+1)
        width = int((width-5)/3+1)
        self.a2 = self.create_conv_layer(self.a1, 3, 2, 256, 128, width)
        width = int((width-3)/2+1)
        width = int((width-3)/2+1)
        self.a3 = self.create_conv_layer(self.a2, 3, 2, 256, 256, width)      
        width = int((width-3)/2+1)
        width = int((width-3)/2+1)
        self.flatten =  tf.reshape(self.a3, [-1, width*256])
        
        self.dense1 = tf.layers.dense(inputs=self.flatten, units=int(width/2),
                                      activation=tf.nn.relu, name='dense1')
        self.do1 = tf.nn.dropout(self.dense1, 0.5)
        self.dense2 = tf.layers.dense(inputs=self.do1, units=int(width/4),
                                      activation=tf.nn.relu, name='dense2') 
        self.do2 = tf.nn.dropout(self.dense2, 0.5)
        self.dense3 = tf.layers.dense(inputs=self.do2, units=20,
                                      activation=tf.nn.sigmoid, name='logits')
        
        self.softmax = tf.nn.softmax(self.dense3, name="softmax_tensor")
        self.predictions = tf.argmax(self.softmax, 1, name="predictions")
        
        losses = tf.nn.softmax_cross_entropy_with_logits(labels=self.Y, logits=self.dense3)
        self.loss = tf.reduce_mean(losses)
        
        correct_predictions = tf.equal(self.predictions, tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
        
    def create_conv_layer(self, input_tensor, kernel_shape=3, stride=1, 
                          n_filters=64, in_channels=1, input_width=20000):
        with tf.name_scope('convlayer_{}_{}_{}_{}'.format(kernel_shape, stride, n_filters, in_channels)):
            filter_shape = [kernel_shape, in_channels, n_filters]
            W = tf.Variable(tf.random_uniform(filter_shape), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[n_filters]), name="b")
            conv = tf.nn.conv1d(input_tensor, W, stride=stride, padding="VALID", name="conv")
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            
            expected_shape = int((input_width-kernel_shape)/stride+1)
            pooled = tf.nn.max_pool(tf.reshape(h, [-1, expected_shape,n_filters, 1]),
                                    ksize=[1, kernel_shape, 1, 1],
                                    strides=[1, stride, 1, 1],
                                    padding='VALID',
                                    name="pool")
            expected_shape = int((expected_shape-kernel_shape)/stride+1)
            return tf.reshape(pooled,  [-1, expected_shape,n_filters])

    def train(self, sess):

        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(5e-2)
        grads_and_vars = optimizer.compute_gradients(self.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", self.loss)
        acc_summary = tf.summary.scalar("accuracy", self.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpointing
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")

        # Tensorflow assumes this directory already exists so we need to create it
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver()

        tf.global_variables_initializer().run()

        for epoch in range(100):
            train_gen = TextDataSet()
            train_gen.load('class_map.json', 'train.json')

            test_gen = TextDataSet()
            test_gen.load('class_map.json', 'test.json')

            if epoch >0:
                save_path = saver.save(sess, checkpoint_prefix)
                print("Saved model to {}".format(save_path))
            step=0
            for x_batch, y_batch in train_gen:

                feed_dict = {
                  self.X: x_batch,
                  self.Y: y_batch
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, self.loss, self.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                train_summary_writer.add_summary(summaries, step)
                step+=1

                try:
                    x_batch, ybatch = next(test_gen)
                except StopIteration:
                    test_gen = TextDataSet()
                    test_gen.load('class_map.json', 'test.json')
                    x_batch, ybatch = next(test_gen)

                feed_dict = {
                  self.X: x_batch,
                  self.Y: y_batch
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, self.loss, self.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                dev_summary_writer.add_summary(summaries, step)

In [4]:
with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        cnn = TextCNN()
        cnn.train(sess)

Writing to /home/kwierman/Desktop/textrecog/notebook/runs/1516759965



KeyboardInterrupt: 